In [1]:
import requests,json,os,time
import random
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx
import geojson
import ndjson

In [2]:
ecrAppEndpoint = 'https://ecr.sagecontinuum.org/api/apps'
response = requests.request("GET", ecrAppEndpoint)
ecrJson = response.json()

In [3]:
ecrDataDf = pd.DataFrame.from_dict(ecrJson['data'], orient='columns')
ecrDataDf.drop('source',1).join(pd.DataFrame.from_dict(ecrDataDf.source.to_dict(),orient='index'))
ecrDataDf

,arguments,baseCommand,depends_on,description,frozen,id,inputs,metadata,name,namespace,owner,source,time_created,time_last_updated,version
0,,,,Smoke Detection Plugin,False,iperezx/smokedetect:0.5.0,[],{},smokedetect,iperezx,21054217-3dae-4538-9ccb-912e3ae67564,"{'architectures': ['linux/amd64'], 'branch': '...",2021-06-15T21:43:49Z,2021-06-15T21:43:49Z,0.5.0
1,,,,Cloud Coverage Estimator,False,sage/cloudcover:0.1.0,"[{'id': 'interval', 'type': 'int'}, {'id': 'sa...",{},cloudcover,sage,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64'], 'branch': '...",2021-05-04T20:07:51Z,2021-05-04T20:07:51Z,0.1.0
2,,,,Car and pedestrian counter,False,waggle/carped:0.0.0,"[{'id': 'interval', 'type': 'int'}]",{'ontology': 'env.counter'},carped,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/arm64'], 'branch': '...",2021-05-04T21:57:41Z,2021-05-04T21:57:41Z,0.0.0
3,,,,my hello world app,False,waggle/helloworld-ml:1.0,"[{'id': 'speed', 'type': 'int'}]",{'my-science-data': 12345},helloworld-ml,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64', 'linux/arm/v...",2021-05-03T20:35:11Z,2021-05-03T20:35:11Z,1.0
4,,,,Image sampler,False,waggle/image-sampler:0.1.0,"[{'id': 'interval', 'type': 'int'}, {'id': 'st...",NaN,image-sampler,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64', 'linux/arm64...",2021-04-30T19:46:41Z,2021-04-30T19:46:41Z,0.1.0
5,,,,Plugin which uses BME680 sensor to provide bas...,False,waggle/metsense:0.0.1,[],{},metsense,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64', 'linux/arm/v...",2021-05-04T20:12:43Z,2021-05-04T20:12:43Z,0.0.1
6,,,,my hello world app,False,wgerlach/helloworld-ml:1.0,"[{'id': 'speed', 'type': 'int'}]",{'my-science-data': 12345},helloworld-ml,wgerlach,c38d37f8-d274-11e5-b5b1-571c90c58959,"{'architectures': ['linux/amd64'], 'branch': '...",2021-05-20T19:21:24Z,2021-05-20T19:21:24Z,1.0
7,,,,my hello world app,False,wgerlach/helloworld-ml-test2:1.0,"[{'id': 'speed', 'type': 'int'}]",{'my-science-data': 12345},helloworld-ml-test2,wgerlach,c38d37f8-d274-11e5-b5b1-571c90c58959,"{'architectures': ['linux/amd64', 'linux/arm/v...",2021-06-08T00:46:56Z,2021-06-08T00:46:56Z,1.0


In [5]:
exceptColmns = ['id']
suffix = '_ecr'
ecrDataDf.columns = ecrDataDf.columns.map(lambda x : x+suffix if x not in exceptColmns else x)
ecrDataDf.rename(columns={'id':'pluginID'},inplace=True)

In [6]:
ecrDataDf

,arguments_ecr,baseCommand_ecr,depends_on_ecr,description_ecr,frozen_ecr,pluginID,inputs_ecr,metadata_ecr,name_ecr,namespace_ecr,owner_ecr,source_ecr,time_created_ecr,time_last_updated_ecr,version_ecr
0,,,,Smoke Detection Plugin,False,iperezx/smokedetect:0.5.0,[],{},smokedetect,iperezx,21054217-3dae-4538-9ccb-912e3ae67564,"{'architectures': ['linux/amd64'], 'branch': '...",2021-06-15T21:43:49Z,2021-06-15T21:43:49Z,0.5.0
1,,,,Cloud Coverage Estimator,False,sage/cloudcover:0.1.0,"[{'id': 'interval', 'type': 'int'}, {'id': 'sa...",{},cloudcover,sage,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64'], 'branch': '...",2021-05-04T20:07:51Z,2021-05-04T20:07:51Z,0.1.0
2,,,,Car and pedestrian counter,False,waggle/carped:0.0.0,"[{'id': 'interval', 'type': 'int'}]",{'ontology': 'env.counter'},carped,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/arm64'], 'branch': '...",2021-05-04T21:57:41Z,2021-05-04T21:57:41Z,0.0.0
3,,,,my hello world app,False,waggle/helloworld-ml:1.0,"[{'id': 'speed', 'type': 'int'}]",{'my-science-data': 12345},helloworld-ml,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64', 'linux/arm/v...",2021-05-03T20:35:11Z,2021-05-03T20:35:11Z,1.0
4,,,,Image sampler,False,waggle/image-sampler:0.1.0,"[{'id': 'interval', 'type': 'int'}, {'id': 'st...",NaN,image-sampler,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64', 'linux/arm64...",2021-04-30T19:46:41Z,2021-04-30T19:46:41Z,0.1.0
5,,,,Plugin which uses BME680 sensor to provide bas...,False,waggle/metsense:0.0.1,[],{},metsense,waggle,17c1a10d-3a8a-40ab-b0b8-43d56cafef1c,"{'architectures': ['linux/amd64', 'linux/arm/v...",2021-05-04T20:12:43Z,2021-05-04T20:12:43Z,0.0.1
6,,,,my hello world app,False,wgerlach/helloworld-ml:1.0,"[{'id': 'speed', 'type': 'int'}]",{'my-science-data': 12345},helloworld-ml,wgerlach,c38d37f8-d274-11e5-b5b1-571c90c58959,"{'architectures': ['linux/amd64'], 'branch': '...",2021-05-20T19:21:24Z,2021-05-20T19:21:24Z,1.0
7,,,,my hello world app,False,wgerlach/helloworld-ml-test2:1.0,"[{'id': 'speed', 'type': 'int'}]",{'my-science-data': 12345},helloworld-ml-test2,wgerlach,c38d37f8-d274-11e5-b5b1-571c90c58959,"{'architectures': ['linux/amd64', 'linux/arm/v...",2021-06-08T00:46:56Z,2021-06-08T00:46:56Z,1.0
